# Test Concepts Processor

**Purpose:** Extract test concepts from Excel sheets and store as `transfer_tuple_map` (MAP<STRING, STRING>).

**Pattern:** Uses the same scalable `mapInPandas` pattern as transfer variables processor.

**IMPORTANT:** Test concepts are **OPTIONAL**. This processor will NOT fail the job if:
- No test_concepts sheets are found
- Processing errors occur (logged but continue)
- No test concepts are extracted

**Algorithm:**
1. Read test_concepts sheets from `md_dta_excel_file_raw` (anti-join for incremental processing)
2. Join with `md_file_history` to get Excel content
3. Load transfer variables for header detection (from silver table)
4. Use `mapInPandas` for parallel processing across Spark workers
5. Create `transfer_tuple_map` for each data row (key=column header, value=cell value)
6. Compute `definition_hash` from the entire tuple map
7. Save to `silver_md.md_dta_vendor_test_concepts_draft`

**Key Design:**
- Uses `mapInPandas` for scalable parallel processing (same pattern as transfer variables)
- No fixed schema for test concepts - uses MAP<STRING, STRING> for format-tolerance
- Header detection uses existing transfer variable names as reference
- Handles heterogeneous vendor formats automatically
- Schema evolution enabled via `mergeSchema=true`
- Graceful error handling - sets status and notes for downstream tasks


In [ ]:
# Cell 1: Imports
import os
import uuid
import json
import base64
import hashlib
import re
import pandas as pd
from io import BytesIO
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_timestamp
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, 
    BooleanType, ArrayType, MapType, TimestampType, DoubleType
)
from delta.tables import DeltaTable

print("="*80)
print("TEST CONCEPTS PROCESSOR (mapInPandas Pattern)")
print("="*80)


In [ ]:
# Cell 2: Check if Previous Task Found Documents
# NOTE: Test concepts are OPTIONAL - we skip gracefully if no documents
try:
    documents_found = dbutils.jobs.taskValues.get(taskKey="extract_excel_sheets", key="documents_found")
    
    if documents_found == "false":
        print("⏭️  No documents were found in previous task (extract_excel_sheets)")
        print("   Skipping test concepts processing (test concepts are optional).")
        dbutils.jobs.taskValues.set(key="test_concepts_count", value="0")
        dbutils.jobs.taskValues.set(key="test_concepts_status", value="SKIPPED")
        dbutils.jobs.taskValues.set(key="test_concepts_notes", value="No documents found by extract_excel_sheets")
        dbutils.notebook.exit("Skipped - no documents to process")
    
    print(f"✓ Documents found in previous task: {documents_found}")
    
except Exception as e:
    # If task value doesn't exist, assume we should continue (backwards compatibility)
    print(f"⚠ Could not read 'documents_found' task value: {e}")
    print("  Continuing anyway (backwards compatibility)")
    pass


In [ ]:
# Cell 3: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

# Extract needed values from globals
catalog = globals_dict['catalog']
bronze_schema = globals_dict['bronze_schema']
silver_schema = globals_dict['silver_schema']

print(f"Catalog: {catalog}")
print(f"Bronze Schema: {bronze_schema}")
print(f"Silver Schema: {silver_schema}")
print(f"Created by: {created_by_principal}")
print(f"Job ID: {databricks_job_id}, Job Name: {databricks_job_name}, Run ID: {databricks_run_id}")


In [ ]:
# Cell 4: Define Table Names
history_table = f"{catalog}.{bronze_schema}.md_file_history"
sheets_table = f"{catalog}.{bronze_schema}.md_dta_excel_file_raw"
transfer_vars_table = f"{catalog}.{silver_schema}.md_dta_transfer_variables_draft"
output_table = f"{catalog}.{silver_schema}.md_dta_vendor_test_concepts_draft"

print(f"History Table: {history_table}")
print(f"Sheets Table: {sheets_table}")
print(f"Transfer Variables Table: {transfer_vars_table}")
print(f"Output Table: {output_table}")


In [ ]:
# Cell 5: Read Test Concepts Sheets with Parent Content
# Join sheet metadata with parent document content (base64)
# Use anti-join for incremental processing
# NOTE: Test concepts are OPTIONAL

print(f"{'='*80}")
print(f"Reading test_concepts sheets...")
print(f"{'='*80}")

# Read sheet metadata filtered for test_concepts category
df_sheets = spark.table(sheets_table).filter(
    F.col("sheet_category") == "test_concepts"
).select(
    "document_id",
    "parent_document_id",
    "sheet_name",
    "source_file_path",
    "trial_id",
    "data_stream_type",
    "data_provider_name"
)

total_sheets = df_sheets.count()
print(f"Total test_concepts sheets in source: {total_sheets}")

if total_sheets == 0:
    print("⚠ No test_concepts sheets found in any documents")
    print("  This is valid - test concepts are optional")
    dbutils.jobs.taskValues.set(key="test_concepts_count", value="0")
    dbutils.jobs.taskValues.set(key="test_concepts_status", value="NO_DATA")
    dbutils.jobs.taskValues.set(key="test_concepts_notes", value="No test_concepts sheets found in any documents")
    dbutils.notebook.exit("Completed - no test_concepts sheets found (test concepts are optional)")

# Check which parent documents have already been processed
if spark.catalog.tableExists(output_table):
    df_already_processed = spark.table(output_table).select(
        F.col("parent_document_id")
    ).distinct()
    
    # Anti-join to exclude sheets from already processed parent documents
    df_sheets_to_process = df_sheets.join(
        df_already_processed,
        on="parent_document_id",
        how="left_anti"
    )
    
    already_processed_count = total_sheets - df_sheets_to_process.count()
    print(f"Sheets from already processed documents: {already_processed_count}")
    print(f"Sheets from new documents to process: {df_sheets_to_process.count()}")
else:
    print(f"Output table does not exist yet - will create it")
    df_sheets_to_process = df_sheets

# Join with parent documents to get content_base64
df_with_content = df_sheets_to_process.join(
    spark.table(history_table).filter(
        F.col("is_current") == True
    ).select(
        F.col("document_id").alias("parent_id"),
        "content_base64"
    ),
    df_sheets_to_process.parent_document_id == F.col("parent_id")
)

sheet_count = df_with_content.count()
print(f"Sheets to process in this run: {sheet_count}")
print(f"{'='*80}")

if sheet_count == 0:
    print("✓ No new test_concepts sheets to process. All up-to-date.")
    dbutils.jobs.taskValues.set(key="test_concepts_count", value="0")
    dbutils.jobs.taskValues.set(key="test_concepts_status", value="UP_TO_DATE")
    dbutils.jobs.taskValues.set(key="test_concepts_notes", value="All test_concepts sheets already processed")
    dbutils.notebook.exit("All test_concepts sheets already processed")


In [ ]:
# Cell 6: Load Transfer Variables for Header Detection
# Create lookup dictionary keyed by parent_document_id
# This will be serialized to JSON and passed to mapInPandas workers
print(f"{'='*80}")
print("Loading transfer variables for header detection...")
print(f"{'='*80}")

# Get unique parent_document_ids we're processing
parent_doc_ids = [row['parent_document_id'] for row in df_sheets_to_process.select('parent_document_id').distinct().collect()]
print(f"Parent documents to process: {len(parent_doc_ids)}")

# Load transfer variables for these parent documents
transfer_vars_lookup = {}

if spark.catalog.tableExists(transfer_vars_table):
    for parent_doc_id in parent_doc_ids:
        df_tv = spark.sql(f"""
            SELECT DISTINCT 
                transfer_variable_name,
                trial_id,
                data_stream_type,
                data_provider_name
            FROM {transfer_vars_table}
            WHERE parent_document_id = '{parent_doc_id}'
        """)
        
        tv_names = set()
        for row in df_tv.collect():
            if row['transfer_variable_name']:
                tv_names.add(row['transfer_variable_name'].strip().upper())
        
        transfer_vars_lookup[parent_doc_id] = list(tv_names)
        print(f"  {parent_doc_id[:8]}...: {len(tv_names)} transfer variables")
else:
    print(f"⚠ Transfer variables table not found: {transfer_vars_table}")
    print("  Will attempt header detection without transfer variable matching")

# Serialize for distribution to workers
transfer_vars_json = json.dumps(transfer_vars_lookup)
print(f"\n✓ Loaded transfer variables for {len(transfer_vars_lookup)} parent documents")


In [ ]:
# Cell 7: Define Output Schema with Schema Evolution Support
# Uses MAP<STRING, STRING> for transfer_tuple_map to allow different sheets to have different columns

test_concepts_schema = StructType([
    StructField("test_concept_id", StringType(), False),
    StructField("parent_document_id", StringType(), False),
    StructField("dta_id", StringType(), True),
    StructField("trial_id", StringType(), True),
    StructField("data_stream_type", StringType(), True),
    StructField("data_provider_name", StringType(), True),
    StructField("domain_info", StringType(), True),  # Excel sheet name for multi-sheet documents
    StructField("test_concept_reference", StringType(), True),
    StructField("transfer_tuple_map", MapType(StringType(), StringType()), True),
    StructField("codelist_values", ArrayType(StringType()), True),
    StructField("variable_description", StringType(), True),
    StructField("definition_hash", StringType(), True),
    StructField("notes", StringType(), True),
    StructField("status", StringType(), True),
    StructField("vendor_comment", StringType(), True),
    StructField("version", StringType(), True),
    StructField("version_status", StringType(), True),
    StructField("is_current_draft", BooleanType(), True),
    StructField("processing_status", StringType(), True)  # Internal tracking
])

print("✓ Output schema defined with MAP<STRING, STRING> for format-tolerance")


In [ ]:
# Cell 8: Process Test Concepts Using mapInPandas
# Scalable parallel processing across Spark workers
# NOTE: Test concepts are OPTIONAL - errors are logged but don't fail the job

print(f"{'='*80}")
print(f"Processing test concepts sheets with mapInPandas...")
print(f"{'='*80}")

# Add transfer variables JSON as column for distribution to workers
df_with_tv = df_with_content.withColumn("transfer_vars_json", lit(transfer_vars_json))

def process_test_concepts_pandas(pdf_iterator):
    """
    Process test concepts sheets using mapInPandas.
    Each partition processes sheets in parallel across Spark workers.
    
    NOTE: Test concepts are optional - errors are logged but processing continues.
    """
    import pandas as pd
    import json
    import base64
    import hashlib
    import uuid
    import re
    from io import BytesIO
    
    def normalize_header(val):
        """Normalize header value for matching."""
        if val is None or pd.isna(val):
            return ""
        s = str(val).strip().upper()
        s = re.sub(r'\s+', ' ', s)
        return s
    
    def find_header_row_by_transfer_vars(excel_bytes, sheet_name, transfer_variable_names, max_scan_rows=25):
        """
        Find the header row by matching against known transfer variable names.
        Fixed columns (Tab Name, Test Concept Reference) are detected by POSITION.
        """
        try:
            raw_df = pd.read_excel(BytesIO(excel_bytes), sheet_name=sheet_name, header=None, dtype=str)
        except Exception as e:
            return None, {}, {}, str(e)
        
        tv_set = set(transfer_variable_names) if transfer_variable_names else set()
        
        best_row = None
        best_score = 0
        best_tv_map = {}
        
        for row_idx in range(min(max_scan_rows, len(raw_df))):
            row_values = raw_df.iloc[row_idx].values
            tv_map = {}
            
            for col_idx, val in enumerate(row_values):
                normalized = normalize_header(val)
                if normalized and normalized in tv_set:
                    tv_map[col_idx] = normalized
            
            if len(tv_map) > best_score:
                best_score = len(tv_map)
                best_row = row_idx
                best_tv_map = tv_map
        
        if best_row is None or best_score < 3:
            return None, {}, {}, f"Could not find header row (best score: {best_score})"
        
        # Fixed columns are those BEFORE the first transfer variable column
        # Detect dynamically from actual header row values (not hardcoded)
        first_tv_col = min(best_tv_map.keys()) if best_tv_map else 0
        fixed_columns = {}
        
        # Get header row values to read actual column names
        header_values = raw_df.iloc[best_row].values
        for col_idx in range(first_tv_col):
            if col_idx < len(header_values):
                col_name = str(header_values[col_idx]).strip() if pd.notna(header_values[col_idx]) else None
                if col_name:
                    # Normalize to snake_case for consistency
                    normalized_name = col_name.upper().replace(" ", "_").replace("-", "_")
                    fixed_columns[col_idx] = normalized_name
        
        return best_row, best_tv_map, fixed_columns, None
    
    def compute_definition_hash(transfer_tuple_map, data_provider_name, data_stream_type):
        """Compute SHA256 hash of the transfer_tuple_map for deduplication."""
        # Include provider and stream in hash for vendor-specific templates
        hash_dict = {'data_provider': data_provider_name, 'data_stream': data_stream_type, 'tuple_map': transfer_tuple_map}
        sorted_items = sorted(hash_dict.items())
        hash_string = json.dumps(sorted_items, sort_keys=True)
        return hashlib.sha256(hash_string.encode('utf-8')).hexdigest()
    
    for pdf in pdf_iterator:
        results = []
        
        for _, row in pdf.iterrows():
            sheet_name = row['sheet_name']
            parent_document_id = row['parent_document_id']
            content_b64 = row['content_base64']
            trial_id = row['trial_id']
            data_stream_type = row['data_stream_type']
            data_provider_name = row['data_provider_name']
            
            # Parse transfer variables lookup
            try:
                tv_lookup = json.loads(row['transfer_vars_json'])
                transfer_variable_names = tv_lookup.get(parent_document_id, [])
            except:
                transfer_variable_names = []
            
            try:
                # Decode Excel content
                if not content_b64:
                    results.append({
                        'test_concept_id': str(uuid.uuid4()),
                        'parent_document_id': parent_document_id,
                        'dta_id': None,
                        'trial_id': trial_id,
                        'data_stream_type': data_stream_type,
                        'data_provider_name': data_provider_name,
                        'domain_info': sheet_name,  # Excel sheet name
                        'test_concept_reference': f"SHEET:{sheet_name}",
                        'transfer_tuple_map': {},
                        'codelist_values': None,
                        'variable_description': None,
                        'definition_hash': None,
                        'notes': 'No Excel content available',
                        'status': 'MANUAL_REVIEW',
                        'vendor_comment': None,
                        'version': None,
                        'version_status': 'DRAFT',
                        'is_current_draft': True,
                        'processing_status': 'ERROR'
                    })
                    continue
                
                excel_bytes = base64.b64decode(content_b64)
                
                # Find header row
                header_row, tv_map, fixed_cols, error = find_header_row_by_transfer_vars(
                    excel_bytes, sheet_name, transfer_variable_names
                )
                
                if error or header_row is None:
                    # No valid header - create error record
                    results.append({
                        'test_concept_id': str(uuid.uuid4()),
                        'parent_document_id': parent_document_id,
                        'dta_id': None,
                        'trial_id': trial_id,
                        'data_stream_type': data_stream_type,
                        'data_provider_name': data_provider_name,
                        'domain_info': sheet_name,  # Excel sheet name
                        'test_concept_reference': f"SHEET:{sheet_name}",
                        'transfer_tuple_map': {},
                        'codelist_values': None,
                        'variable_description': None,
                        'definition_hash': None,
                        'notes': f"Header detection failed: {error}",
                        'status': 'MANUAL_REVIEW',
                        'vendor_comment': None,
                        'version': None,
                        'version_status': 'DRAFT',
                        'is_current_draft': True,
                        'processing_status': 'ERROR'
                    })
                    continue
                
                # Read data rows
                raw_df = pd.read_excel(BytesIO(excel_bytes), sheet_name=sheet_name, header=None, dtype=str)
                
                # Process data rows (after header)
                for row_idx in range(header_row + 1, len(raw_df)):
                    row_values = raw_df.iloc[row_idx].values
                    
                    # Skip empty rows
                    non_empty = [v for v in row_values if pd.notna(v) and str(v).strip()]
                    if len(non_empty) < 2:
                        continue
                    
                    # Build transfer_tuple_map - ONLY transfer variable columns, in Excel column order
                    transfer_tuple_map = {}
                    
                    # Add transfer variable columns in sorted column order (preserves Excel order)
                    for col_idx in sorted(tv_map.keys()):
                        tv_name = tv_map[col_idx]
                        val = row_values[col_idx] if col_idx < len(row_values) else None
                        transfer_tuple_map[tv_name] = "" if pd.isna(val) else str(val).strip()
                    
                    # Extract fixed column values (NOT added to tuple map, stored as separate fields)
                    fixed_values = {}
                    for col_idx, col_name in fixed_cols.items():
                        val = row_values[col_idx] if col_idx < len(row_values) else None
                        fixed_values[col_name] = None if pd.isna(val) else str(val).strip()
                    
                    # Find test_concept_reference - look for columns with TEST, CONCEPT, or CTESTCD in name
                    test_concept_ref = ''
                    for col_key, col_val in fixed_values.items():
                        if col_val and any(kw in col_key.upper() for kw in ['CTESTCD', 'TEST_CONCEPT', 'TEST_CODE', 'TESTCD']):
                            test_concept_ref = col_val
                            break
                    
                    # Fallback: If no fixed columns or no match, look in transfer variables
                    if not test_concept_ref and transfer_tuple_map:
                        for tv_key, tv_val in transfer_tuple_map.items():
                            if tv_val and any(kw in tv_key.upper() for kw in ['CTESTCD', 'TESTCD', 'TEST_CODE']):
                                test_concept_ref = tv_val
                                break
                    
                    # Last fallback: generate unique reference from row index
                    if not test_concept_ref:
                        test_concept_ref = f"{sheet_name}_ROW{row_idx + 1}"
                    
                    
                    # Compute definition hash
                    definition_hash = compute_definition_hash(transfer_tuple_map, data_provider_name, data_stream_type)
                    
                    # Create base record
                    record = {
                        'test_concept_id': str(uuid.uuid4()),
                        'parent_document_id': parent_document_id,
                        'dta_id': None,
                        'trial_id': trial_id,
                        'data_stream_type': data_stream_type,
                        'data_provider_name': data_provider_name,
                        'domain_info': sheet_name,  # Excel sheet name
                        'test_concept_reference': test_concept_ref,
                        'transfer_tuple_map': transfer_tuple_map,
                        'codelist_values': None,
                        'variable_description': None,
                        'definition_hash': definition_hash,
                        'notes': f"Extracted from sheet: {sheet_name}, row: {row_idx + 1}",
                        'status': 'COMPLETED',
                        'vendor_comment': None,
                        'version': None,
                        'version_status': 'DRAFT',
                        'is_current_draft': True,
                        'processing_status': 'SUCCESS'
                    }
                    # Add fixed columns as dynamic fields (schema evolution with mergeSchema=true)
                    for col_name, col_val in fixed_values.items():
                        # Lowercase for column names
                        record[col_name.lower()] = col_val
                    results.append(record)
                
            except Exception as e:
                # Error processing sheet - create error record
                results.append({
                    'test_concept_id': str(uuid.uuid4()),
                    'parent_document_id': parent_document_id,
                    'dta_id': None,
                    'trial_id': trial_id,
                    'data_stream_type': data_stream_type,
                    'data_provider_name': data_provider_name,
                    'domain_info': sheet_name,  # Excel sheet name
                    'test_concept_reference': f"SHEET:{sheet_name}",
                    'transfer_tuple_map': {},
                    'codelist_values': None,
                    'variable_description': None,
                    'definition_hash': None,
                    'notes': f"Error: {str(e)[:500]}",
                    'status': 'MANUAL_REVIEW',
                    'vendor_comment': None,
                    'version': None,
                    'version_status': 'DRAFT',
                    'is_current_draft': True,
                    'processing_status': 'ERROR'
                })
        
        # Yield results as DataFrame
        yield pd.DataFrame(results)

# Repartition for parallelism
num_partitions = min(max(1, sheet_count // 10), 200)
df_repartitioned = df_with_tv.repartition(num_partitions)
print(f"Using {num_partitions} partitions for parallel processing")

# Execute mapInPandas
df_test_concepts_results = df_repartitioned.mapInPandas(process_test_concepts_pandas, test_concepts_schema)

# Count results
total_count = df_test_concepts_results.count()
success_count = df_test_concepts_results.filter(F.col("processing_status") == "SUCCESS").count()
error_count = df_test_concepts_results.filter(F.col("processing_status") == "ERROR").count()

print(f"\n{'='*80}")
print(f"Processing Results:")
print(f"  Total test concept records: {total_count}")
print(f"  SUCCESS: {success_count}")
print(f"  ERROR: {error_count}")
print(f"{'='*80}")


In [ ]:
# Cell 9: Write Results to Silver Table
# NOTE: Test concepts are OPTIONAL - no results is valid

print(f"\n{'='*80}")
print(f"Writing Results to Silver Table...")
print(f"{'='*80}")

# Handle case where no test concepts found
if total_count == 0:
    print("ℹ️  No test concepts extracted - this is valid (test concepts are optional)")
    dbutils.jobs.taskValues.set(key="test_concepts_count", value="0")
    dbutils.jobs.taskValues.set(key="test_concepts_status", value="NO_DATA")
    dbutils.jobs.taskValues.set(key="test_concepts_notes", value="No test concepts found in sheets")
    dbutils.notebook.exit("Completed - no test concepts extracted (test concepts are optional)")

# Remove processing_status column before writing (internal tracking only)
df_to_write = df_test_concepts_results.drop("processing_status")

# Add audit columns
df_with_audit = df_to_write \
    .withColumn("created_ts", current_timestamp()) \
    .withColumn("created_by_principal", lit(created_by_principal)) \
    .withColumn("last_updated_ts", current_timestamp()) \
    .withColumn("last_updated_by", lit(created_by_principal)) \
    .withColumn("databricks_job_id", lit(databricks_job_id)) \
    .withColumn("databricks_job_name", lit(databricks_job_name)) \
    .withColumn("databricks_run_id", lit(databricks_run_id))

# Write with schema evolution enabled
df_with_audit.write \
    .format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable(output_table)

print(f"✓ Saved {total_count} test concept records to {output_table}")
print(f"  SUCCESS: {success_count}")
if error_count > 0:
    print(f"  ⚠ {error_count} records need manual review")


In [ ]:
# Cell 10: Summary and Task Values
# NOTE: Test concepts are OPTIONAL - status tracking for downstream tasks

print(f"\n{'='*80}")
print("TEST CONCEPTS PROCESSOR - COMPLETED")
print(f"{'='*80}")
print(f"  Sheets Processed: {sheet_count}")
print(f"  Test Concepts Extracted: {success_count}")
print(f"  Processing Errors: {error_count}")
print(f"  Output Table: {output_table}")
print(f"{'='*80}")

# Set task values for downstream tasks
if success_count > 0:
    if error_count > 0:
        task_status = "SUCCESS_WITH_WARNINGS"
        task_notes = f"Extracted {success_count} test concepts. {error_count} records need review."
    else:
        task_status = "SUCCESS"
        task_notes = f"Extracted {success_count} test concepts"
elif error_count > 0:
    task_status = "MANUAL_REVIEW"
    task_notes = f"All {error_count} records need manual review"
else:
    task_status = "NO_DATA"
    task_notes = "No test concepts extracted"

dbutils.jobs.taskValues.set(key="test_concepts_count", value=str(success_count))
dbutils.jobs.taskValues.set(key="test_concepts_status", value=task_status)
dbutils.jobs.taskValues.set(key="test_concepts_notes", value=task_notes)
dbutils.jobs.taskValues.set(key="test_concepts_error_count", value=str(error_count))

print(f"\n✓ Status: {task_status}")
print(f"✓ Notes: {task_notes}")

dbutils.notebook.exit(f"Success: {success_count} test concepts processed")
